# exporting the Breton example sentences

In [17]:
import datetime, tqdm, re, time, string
# !pip install mwclient
# !pip install mwparserfromhell
# !pip install mwcomposerfromhell
from mwclient import Site
#myparserfromhell : parser pour le code wiki
import mwparserfromhell, mwcomposerfromhell
import pickle

In [18]:
print('connecting...')
#site contenant tous les exemples
site = Site('arbres.iker.cnrs.fr/', path='/')
outf = open('problemPages.html','w')
allpages = list(site.allpages())
print('got all', len(allpages), 'pages')

connecting...
got all 9136 pages


#### Charger toutes les pages du site: 

In [19]:
t = time.time()
#dictionnaire contenant toutes les pages aspirées
pages = {}
#on peut ici modifier le nombre de pages à charger pour charger petit à petit ex : allpages[:3000] pour les 3000 premières pages
# newlist=allpages
for p in tqdm.tqdm(allpages):
# for p in tqdm.tqdm(newlist):
    #si les pages ne sont pas de la documentation
    if not('/documentation' in p.name or '/docname' in p.name or 'Module:' in p.name):
        #on ajoute le nom de la page comme clé, son contenu comme valeur
        pages[p.name] = p.text() 
print('extraction done. it took', time.time()-t, 'seconds.')

100%|██████████| 9136/9136 [28:03<00:00,  5.43it/s] 

extraction done. it took 1683.8789749145508 seconds.


#### Compter tous les exemples trouvés dans les pages:

In [7]:
# Pour enregistrer les pages chargées dans un fichier Pickle: 
# une fois toutes les pages chargées, on le met dans un pickle
with open("Pages.pickle", "wb") as myFile:
   pickle.dump(pages, myFile)
with open("Pages.txt", "w") as myFile:
    for title,text in pages.items():
        myFile.write(title+'\n'+text+'\n------------------------\n\n')
    

In [8]:
# print(', '.join(pages.keys()))
#on créé un fichier txt contenant tous les titres de pages
open('pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 9106 pages


In [13]:

with open('Pages.pickle', 'rb') as f:
    pages = pickle.load(f)

In [14]:
import re
repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE) #expression régulière pour trouver les exemples de table, re.MULTILINE pour que le point puisse matcher les retours à la ligne

exampleFile = open('examples.txt','w')
allc=0
# Parcourir toutes les pages et chercher des exemples de tables avec la classe "prettytable"
for title in tqdm.tqdm(pages):
    exampleFile.write(title+'\n_____________________________\n')
    wikitext = pages[title]
    counter = 0
 # Utiliser une expression régulière pour trouver les exemples de table
    for m in repretty.finditer(wikitext): #m est chaque exemple trouvé
        exampleFile.write(m.group(0)+'\n________________\n') #m.group(0) pour matcher la totalité de l'expression régulière
        counter += 1
    # Écrire le nombre d'exemples trouvés dans la page actuelle
    exampleFile.write('\n'+str(counter)+' exemples trouvés dans la page '+title+'\n================================\n\n\n')
    # Ajouter le nombre d'exemples trouvés à la variable totale
    allc+=counter

# Fermer le fichier
exampleFile.close()

# Afficher le nombre total d'exemples
print("Nous avons trouvés", allc,"exemples(y compris doublons)")

100%|██████████| 9106/9106 [00:00<00:00, 21114.63it/s]

Nous avons trouvés 14325 exemples(y compris doublons)


### Nous commençons à extraire des éléments sur les exemples

In [ ]:
"""
Exemple de tableau standard (4 lignes):

{| class="prettytable"
|(1)|| N'eo || ket || dleet || koduiñ || hag || evañ || er || memes || amzer.
|-
||| [[ne]] [[COP|est]] || [[ket|pas]] || [[dleout|dû]] || conduire || [[&|et]] || [[evañ|boire]] || [[P.e|en]].[[art|le]] || [[memes|même]] || temps
|-
|||colspan="10" | 'On ne doit pas boire et conduire en même temps /Il ne faut pas boire et conduire' 
|-
|||||||||colspan="10" | ''Lesneven/Kerlouan'', [[Y. M. (04/2016)]]
|}
"""
#dictionnaire des dialectes à récupérer
dialects = {"léonard" : [], "cornouaillais" : [], "trégorrois": [], "vannetais" : [], "standard" : [], "inconnu" : [],
            "breton central" : []}
references = {} # keeps track of the references of the sentences: ref:[sentence]
sentences = {} # keeps the actual texts
dic = {} # token to lemma, pos, gloss
lispos = {} 
#set pour éviter les doublons
unique_sentences = set()
currentPage=''


## OUVERTURE DES FICHIERS
#fichier d'erreur
errorout = open('breton.errors.txt','w', encoding='utf-8')
bilan = open('bilan.txt','w', encoding='utf-8')
#fichier contenant les tableaux qu'on ne veut pas traiter
non_examples = open('non_examples.txt','w', encoding='utf-8')


### EXPRESSIONS REGULIERES
repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE)
rePOS = re.compile(r'verbes|auxiliaires|copules|adverbes|complémenteurs|conjonctions|prépositions|adjectifs|noms|particules verbales|interjections|postpositions|déterminants|quantifieurs|pronoms|noms propres|suffixe|interrogatifs|préfixes|modaux|pluriels|indéfinis|particules de discours|finales|exclamatifs', re.IGNORECASE)
reDialect = re.compile(r'léonard|cornouaillais|trégorrois|vannetais|breton central|standard', re.IGNORECASE)
reLanguage = re.compile(r"basque|français de Basse-Bretagne|franco-breton|gaulois|gallo|tohono o'odham|chalcatongo mixtec|tchèque|gallois|roumain|italien|espagnol|hébreu|arabe|français|anglais|allemand|moyen breton|vieux breton|breton pré-moderne", re.IGNORECASE)
reType = re.compile(r'ouvrages de recherche|références de corpus|élicitations|ouvrages pédagogiques|dictionnaires|grammaires', re.IGNORECASE)
reNoms = re.compile(r'Yann|Marijo|Anna|Angela|Alejandro|Odile|Anastazi|Nolwenn|Solange|Bastien|Soazig|Marie|Marsel|Lukaz|Lenaig|Kristof|Julie|Jenovefa|Simone|Mona|Mari', re.IGNORECASE)
relinks = re.compile(r"\[\[(.*?)\]\]")
relinks2 = re.compile(r"\[\[.*\]\]") 
reAlternatives = re.compile(r'\([^\W\d_].* */ *[^\W\d_].*\)', re.U)
reWord = re.compile(r'\[\[[a-z]+')
nospace2 = re.compile(r".*(-.*\(.*)")
nospace3 = re.compile(r".*(.*-,.*-).*")


#éléments de la glose dont on veut supprimer les espaces (pour éviter de splitter sur les espaces)
nospace = {"numéraux cardinaux": "numéraux_cardinaux","nom propre": "nom_propre" , "pronom incorporé": "pronom_incorporé" ,
"nom de titre": "nom_de_titre", 'pronom réfléchi': "pronom_réfléchi", 
"nom nu": "nom_nu", "pluriel interne": "pluriel_interne", "Les pronoms d'@incise contrastifs": "Les_pronoms_d'@incise_contrastifs", 
"particule o": "particule_o", "auxiliaire ober": "auxiliaire_ober" , "IMP on": "IMP_on",
"bezañ préverbal": "bezañ_préverbal", "Nep X":  "Nep_X", "Objet postverbal d'@un infinitif": "Objet_postverbal_d'un_infinitif", 
"Objet postverbal avec le verbe '@avoir": "Objet_postverbal_avec_le_verbe_'avoir", 
"Indéfini de choix libre par reduplication": "Indéfini_de_choix_libre_par_reduplication", 
"châtaigne électrique": "châtaigne_électrique", "numéral ordinal": "numéral_ordinal",
"avant de": "avant_de", "Pronom réfléchi": "Pronom_réfléchi", "Gwazh, gwashañ": "Gwazh,_gwashañ",
"jours de la semaine" : "jours_de_la_semaine", "jour de la semaine": "jour_de_la_semaine" } 

#catégories renseignées entre parenthèses dans la glose. Ex: -et(Adj.)
categories = {"V": "verbe", "Adj": "adjectif", "PL": "suffixe", "N": "nom", "Adv": "adverbe"}
# dictionnaire de correspondance pour trouver les dialectes dans les lignes source: Léon --> Léonard 
regions = {"Léon": "Léonard", "Cornouailles": "Cornouaillais", "Vannes": "Vannetais", "Trégor": "Trégorrois" }
# regex pour supprimer les espaces dans les interjections dans la glose
interjections = re.compile("Eh bien quoi|Ma parole|eh bien", re.IGNORECASE)
# Rannig + verbe avoir à ne pas séparer 
one_token = ["am-oa","he-devoa", "en em", "on-oa", "em-eus", "am-eus", "e-nije", "e-nevoa", "he-doa", 
             "en ur", "'m eus", "em-bije", "en-deus", "o-deus", "em-bije", "em-oa", "am-bez", "am-bo", "o-devoa", "o-devez", "o-deveze",
                        "o-doa"]

                        
#compteurs: nombre de tableaux trouvés, nombre de tableaux dans le fichier d'erreur, nombre de tableaux à ne pas traiter, 
#nombre de tableaux traités et nombre de doublons
total_example = 0
error_count = 0
other_table = 0
converted_tables = 0
doublon = 0

In [ ]:
# getFirstPos: RECHERCHE DE POS DANS UNE PAGE TITRE 
# la fonction cherche le pos dans toutes les pages selon le lemme donné et retourne le pos associé
# si aucun pos n'est trouvé, on met un point d'interrogation
def getFirstPos(title, gloss, currentPage):
    mpos=''  
    if not title: 
        return '?' 
    elif title.isupper() and title != 'R':
        mpos = title  
    # si on cherche la Page "a", on aura une catégorie erronée (préposition au lieu de particule verbale) 
    elif title == 'a' or title == 'e' and 'R' in gloss :
        mpos='particule_verbale'
    elif title == 'P.e' or title == 'P/e':     
        mpos='préposition'
    # on vérifie si la catégorie est présente dans la glose
    elif re.match(r".*\((.*\.)\)", title) and re.match(r".*\((.*\.)\)", title).group(1) in categories:                    
        test_pos = re.match(r".*\((.*\.)\)", title).group(1)                   
        if test_pos.replace(".", "") in categories:                       
            mpos = categories[test_pos.replace(".", "")]                 
    else:        
    #si le titre contient un tiret bas, on le remplace par un espace pour le trouver dans les titres de pages
        if '_' in title: 
            title = title.replace("_", " ")
        if '#' in title: 
            title = title.replace("#", "-")            
#sinon on met une majuscule à la première lettre du titre
        else:             
            title = title[0].upper()+title[1:]
    #si le titre est contenu dans le dictionnaire pages
        if title in pages:           
            wikicode=''
            # on traite les redirections vers d'autres pages
            if '#REDIRECTION' in pages[title]:             
                #on suit la redirection
                newtitle = relinks.search(pages[title]).group(1)                
                newtitle = newtitle[0].upper()+newtitle[1:]
                if '_' in newtitle: 
                    newtitle = newtitle.replace("_", " ")            
                #et on regarde à nouveau si le titre est dans le dictionnaire pages
                if newtitle in pages:
                    wikicode = pages[newtitle]              
                else: 
                    newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
                    if newtitle in pages:
                        wikicode = pages[newtitle]
                    else:
                        newtitle = newtitle.split(',')[0]
                        newtitle = newtitle[0].upper()+newtitle[1:]
                        if newtitle in pages:
                            wikicode = pages[newtitle]
                        else:
                            errorout.write('__currentPage:\nstrange redirect to page that does not exist: '+newtitle+'\n')
                            error_count += 1                            
            else:
                wikicode = pages[title]   
            # on cherche les catégories renseignées sur la page
            if re.findall(r"\[\[Category:.*\|", wikicode):
                cats = re.findall(r"\[\[Category:.*\|", wikicode)               
                list_cats = set()
                for cat in cats:   
                    # on cherche les POS dans les catégories          
                    if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
                        list_cats.add(rePOS.search(cat).group(0).rstrip('s'))                    
                        if len(list_cats) == 1:                                          
                            mpos = list(list_cats)[0]                         
                        elif len(list_cats) >0: 
                            mpos = "/".join(list_cats)    
                    else: 
                        mpos="?"                              
            if mpos:              
                mpos = mpos
            else:
                mpos = "?"
        else:          
            if reNoms.search(title):             
                mpos = "nom_propre"
            # lorsqu'on se trouve sur la page du mot, le mot est entourés de trois guillemets (remplacés par %)
            # on cherche les POS sur la page où on se trouve
            elif "%" in title:             
                wikicode = pages[currentPage]                               
                if re.findall(r"\[\[Category:.*\|", wikicode):
                    cats = re.findall(r"\[\[Category:.*\|", wikicode)
                    list_cats = set()
                    for cat in cats:
                        if cat not in lispos: 
                            lispos[cat] = 0
                        else: 
                            lispos[cat] += 1                            
                        if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
                            list_cats.add(rePOS.search(cat).group(0).rstrip('s'))
                            if len(list_cats) == 1:                                          
                                mpos = list(list_cats)[0]
                            elif len(list_cats) >0: 
                                mpos = "/".join(list_cats)                               
            else:               
                mpos = "?"              
    return mpos

In [ ]:
# Suppression des espaces dans les [[ ]] sinon, le split avec espace comme délimiteur espace va mal se passer...
# case : [[koll|perd]].[[-et (Adj.)|u]]
def sansespace(ch):
    for e in relinks.finditer(ch):
        if " " in e.group(0):
            start, end = e.start(), e.end()
            newch = re.sub(r"\[\[(.*?) (.*?)\]\]", "[[\\1_\\2]]", e.group(0))
            newch = newch.replace(' ', '_') 
            ch = ch[:start] + newch + ch[start+len(newch)+1:]
    return ch
#exemple :
#print(sansespace("[[art|le]] _[[1]]_[[hey j]]_[[maouez | femme]]"))
#resultat : "[[art|le]] _[[1]]_[[heyj]]_[[maouez| femme]]")
#print(sansespace("[[R]] [[+C]] [[ez eus|est]]"))

In [ ]:
## REGEX POUR LES MUTATIONS ET CONSONNES EPENTHETIQUES
mutation1 = re.compile(r"_\[\[([12345])\]\]_\[\[(.*)\]\]")
mutation2 = re.compile(r"\[?\[?(.*)\]?\]?_\[\[([12345])\]\]_")

#ex: "[[ne]]_[[+C]]_"
epenthetique1 = re.compile(r"\[\[(.*)\]\]_,?/?(\[?\[?\+C\]?\]?)_")
#ex: [[ma|que]]_[[4]],+C_
epenthetique2 = re.compile(r"\[\[(.*)\]\]_\[\[([12345])\]\],?/?(\[?\[?\+C\]?\]?)_")
#ex: [[R]]_[[+C]],[[4]]_  
epenthetique3 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?),\[\[([12345])\]\]_")
#ex : [[R]]_[[+C]]_[[ez eus|est]]
#epenthetique4 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?)_[\[(.*)\]\]") 
mutation = False
epenthetique = False

groupes_pipe = re.compile(r"(.*?)\|(.*)")



# tokentrans2lemposglossmorph: FONCTION QUI PREND UN TOKEN ET RETOURNE LEMME, POS, GLOSE
# NB : tr = traduction ; t = token

def tokentrans2lemposglossmorph(tr, t, title, currentPage):
    global error_count
    # print(f't:{t}, tr:{tr}')         
    lem, pos, gloss, morph = '', '', '', ''
    m = relinks.search(tr)     
    test_mutation1 = mutation1.search(tr)
    test_mutation2 = mutation2.search(tr)
    test_epenthetique1 = epenthetique1.search(tr)  
    test_epenthetique2 = epenthetique2.search(tr)  
    test_epenthetique3 = epenthetique3.search(tr)
    

    if test_mutation1 :     
        mutation = True        
        if ('.[[' or ']].' or '-[[' or ']]-' ) in tr:      
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')                 
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]]             
            if re.search(r"\[ ?- ?[A-Za-z]", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):         
                test = tr.split()[i]           
                if relinks2.search(test):                   
                    ele = relinks2.search(test).group(0)                                        
                    lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                    lems += [lem]
                    glosss += [gloss]
                    poss += [getFirstPos(lem, gloss, currentPage)]
                else:                                  
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                        error_count += 1
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."):
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    lemt.append(lems[i].replace('#', '-')) 
                    glosst.append(glosss[i])                    
                    post.append(poss[i]) 
        
        else:         
            morph = test_mutation1.group(1) + " " + "target"
            token_mute = test_mutation1.group(2)       
            token_mute_pipe = groupes_pipe.search(token_mute)
            if token_mute_pipe: 
                lem = token_mute_pipe.group(1)
                gloss = token_mute_pipe.group(2)
                if gloss == 'R':  
                    pos = "particule_verbale" 
                else:   
                    pos = getFirstPos(lem, gloss, currentPage)
            else :
                lem = t.lower()        
                gloss = token_mute   
                if gloss == 'R'  or '[[R]' in gloss:  
                    pos = "particule_verbale" 
                else:                          
                    pos = getFirstPos(t, gloss, currentPage)
                if pos == '?':
                    pos = getFirstPos(tr, gloss, currentPage)
                    
                    
    elif test_mutation2 :     
        mutation = True        
        if '.[[' or ']].' or '-[[' or ']]-' in tr:            
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')    
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]]             
            if re.search(r"\[ ?- ?[A-Za-z]", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):                
                test = tr.split()[i]  
                if relinks2.search(test):                 
                    ele = relinks2.search(test).group(0)                                        
                    try: 
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:                                   
                            errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                            error_count += 1
                else:                                  
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}") 
                        error_count += 1
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."): 
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    try:                     
                        lemt.append(lems[i].replace('#', '-')) 
                        glosst.append(glosss[i])                    
                        post.append(poss[i])
                    except:                        
                        errorout.write(f"__{currentPage}:\nErreur dans les morphèmes du mot suivant : {tr} et {t}")
                        error_count += 1
        
        else:              
            morph = test_mutation2.group(2) + " " + "trigger"
            token_mute = test_mutation2.group(1)       
            token_mute_pipe = groupes_pipe.search(token_mute)          
            if token_mute_pipe:      
                lem = token_mute_pipe.group(1)
                gloss = token_mute_pipe.group(2)
                if gloss == 'R'  or '[[R]' in gloss:  
                    pos = "particule_verbale" 
                else:                
                    pos = getFirstPos(lem, gloss, currentPage)
            else :
                lem = t.lower() 
                gloss = token_mute
                pos = getFirstPos(t, gloss, currentPage)
                if pos == '?':
                    pos = getFirstPos(tr, gloss,  currentPage)


#S'il y a une consonne épenthétique (+C)             
    elif test_epenthetique1 : 
        epenthetique = True    
        morph = test_epenthetique1.group(2) 
        token_mute = test_epenthetique1.group(1)       
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)

    elif test_epenthetique2 :
        epenthetique = True    
        morph = test_epenthetique2.group(2) + '|' +  test_epenthetique2.group(3) 
        token_mute = test_epenthetique2.group(1)    
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)
                             
    elif test_epenthetique3:
        epenthetique = True 
        morph = test_epenthetique3.group(2) + '|' +  test_epenthetique3.group(3)
        token_mute = test_epenthetique3.group(1)       
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)                    
                               
# si ce n'est pas une mutation et si on a des double crochets [[ ]]              
    elif m:     
        # cas "amalgames" et/ou mots contenant un tiret: 
        #exemple [[da|à]].[[pronom incorporé|vous]] ou [[den|parent]].[[pluriel interne|s]]-[[kozh|vieil]]  
        if '.[[' in tr or ']].' in tr or '-[['in tr or ']]-' in tr: 
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')      
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]] 
            if re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):         
                test = tr.split()[i]                 
                if relinks2.search(test):                 
                    ele = relinks2.search(test).group(0)                                        
                    lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                    lems += [lem]
                    glosss += [gloss]
                    poss += [getFirstPos(lem, gloss, currentPage)]
                else:    
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:              
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                        error_count += 1 
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."): 
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    lemt.append(lems[i].replace('#', '-')) 
                    glosst.append(glosss[i])                    
                    post.append(poss[i])                                        
            lem = ''.join(lemt)
            gloss = ''.join(glosst)
            pos = ''.join(post)             
            return lem, pos, gloss, morph        
        
        
        
        # cas "normal" : pas d'amalgame, un token et une gloss --> [[token|gloss]]
        if '|' in m.group(1):          
            if "numéraux_cardinaux" in m.group(1): 
                lem = t.lower() 
                pos = m.group(1).split('|')[0]
                gloss = m.group(1).split('|')[1] 
            else:                 
                lem = m.group(1).split('|')[0]
                gloss = m.group(1).split('|')[1]                  
                if gloss == 'R' or '[[R]' in gloss:  
                    pos = "particule_verbale"                      
                elif lem == 'P.e' or lem == 'P/e':  
                    lem = "e"
                    pos = "préposition"           
                elif lem == 'nom_propre':  
                    pos = lem
                    lem = gloss                 
                else:
                    pos = getFirstPos(lem, gloss, currentPage)                        
                if pos=='?':
                    pos = getFirstPos(t, gloss, currentPage)
                
        # cas sans amalgame mais sans gloss --> [[ne]]
        else:           
            lem = t
            gloss = relinks.search(tr).group(1)
            if gloss == 'R'  or '[[R]' in gloss:  
                pos = "particule_verbale"
            if 'DIM' in gloss and t.endswith('ig') or t.endswith('ig%'):
                lem = "-ig"                
                pos = "suffixe"                
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)      
    # cas d'un mot seul sans crochets --> foot dans la page "Abardaez,_enderv"      
    else :
        if "%" in t and t.replace("%", "").startswith(currentPage.lower()): 
            lem = currentPage.lower()        
        else: 
            lem = t
        gloss = tr
        if gloss == 'R'  or '[[R]' in gloss:  
            pos = "particule_verbale"            
        else:
            pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)
    return lem, pos, gloss, morph

In [ ]:
# Nettoyage des tokens + remplacement de certains caractères par d'autres
# pour éviter des erreurs quand on splitte
def cleanToken(t):
    t = t.replace("...", "…")   
    t = t.replace("tout.à.l'heure", "tout_à_l'heure")    
    t = t.replace("'''", "%")
    t = t.replace("''", "")
    t = re.sub(r'</?font.*?>','',t)
    t = re.sub(r'<sup>','_',t)
    t = re.sub(r'</sup>','_',t)
    t = re.sub(r'<u>','',t)
    t = re.sub(r'</u>','',t)
    t = re.sub(r'<sub>.*?</sub>','',t)
    t = re.sub(r'\(\[\[\*\]\].*?\)','$$$',t)
    t = re.sub(r"c'h",'cxxxh',t)
    t = re.sub(r"C'h",'Cxxxh',t)
    t = re.sub(r"`",'',t) 
    t = re.sub(r"#",'-',t)   
    # pour spliter sur les apostrophes sans les enlever : mettre @ pour spliter sur @ et donc conserver l'apostrophe
    if not t.strip().startswith("'"):   
        t = re.sub(r"'","'@",t)
    t = re.sub(r"bez'@","bez'",t) 
    t = re.sub(r"Bez'@","B ez'",t)     
    t = re.sub(r"d'@ici","d'ici",t)   
    t = re.sub("aujourd'@hui", "aujourd'hui", t)  
    t = re.sub("l'@heure", "l'heure", t)
    t = re.sub("si.ce.n'@est", "si.ce.n'est", t)   
    t = re.sub("aujourd'@hui", "aujourd'hui", t)      
    return t.strip()
    # Remettre les tokens qui ont été remplacés
def remettretoken(t):
    t = re.sub(r"%","",t)
    t = re.sub(r'cxxxh',"c'h",t)
    t = re.sub(r'Cxxxh',"C'h",t)
    t = re.sub(r"'@", "'",t)
    t = re.sub(r"\$\$\$ ", "",t)
    t = re.sub(r"\[\[", "",t)   
    t = re.sub(r"\]\]", "",t) 
    t = t.replace("..", ".")     
    t = t.replace("e'@idin", "e'idin")          
    return t

# Savoir si une string contient un nombre
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


In [ ]:
## analyzePage: FONCTION PRINCIPALE
def analyzePage(title): 
    global total_example, error_count, other_table, converted_tables, problem
    currentPage = title
    mpos = rePOS.search(pages[title])
    pagepos = None
    conlls=[]
    grammatical = True
    comparison = False  
    example = True
    s= ''
    if mpos:
        # met tout en minuscule
        pagepos = mpos.group(0).lower()
    #pour chaque tableau (et donc chaque exemple)
    nb_tables = 0    
    for m in repretty.finditer(pages[title]):
        total_example += 1        
        prettytable = m.group(1)        
        if "font color=white" in prettytable: 
            example = False
            non_examples.write("__"+currentPage+"\nAutre type de tableau:\n"+ prettytable + "\n\n")            
            break      
        nb_tables += 1        
        tokens, trans = [], []
        translation, source, source2 = '', '', ''
        location, dialect, phonetic, texttype, l = '','','', '', ''
        grammatical=True         

        # si le tableau est "standard" avec une ligne de token, une ligne de glose et une ligne source (9 lignes dans le texte wiki)
        if (len(m.group(1).split('\n'))) == 9 :
            for li in m.group(1).split('\n'): # for every line of the table              
                if li[:2]=='|(' :                   
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')                       
                    if '[[*]]' in li.split('||')[0]:
                        grammatical = False                     
                    tokens = [cleanToken(t) for t in li.split('||')[1:]]  
                    if "vs." in tokens: 
                        comparison = True                        
                    if tokens[0]=="[[*]]" :
                        grammatical = False   
                    elif '[' in tokens[0] and not "[…]" in tokens[0]:                     
                        tokens[0] = tokens[0].replace('[', '').strip()     
                    elif ']' in tokens[-1] and not '[…]' in tokens[-1]: 
                        tokens[-1] = tokens[-1].replace(']', '').strip()
                    if '' in tokens: 
                        while '' in tokens: 
                            tokens.remove('')         
                                                                           
                if li[:3]=='|||' and grammatical:
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    if not trans: #la première fois = la traduction                      
                        trans = [cleanToken(tr) for tr in li[3:].split('||')]                            
                        if '' in trans: 
                            while '' in trans: 
                                trans.remove('')                                
                        if "Glose en [[KLT']" in trans[-1]:
                            trans[-1] = trans[-1].replace("Glose en [[KLT']", '').strip()       
                        if 'Graphie peurunvan' in trans[-1]: 
                            trans[-1] = trans[-1].replace('Graphie peurunvan', '').strip()                           
                        for i in range(len(trans)):                         
                            for ele in nospace:                                
                                if ele in trans[i]: 
                                    trans[i]= trans[i].replace(ele, nospace[ele]) 
                            if re.search(nospace2, trans[i]):
                                to_replace = re.search(nospace2, trans[i]).group(1)
                                trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", "_")) 
                            if re.search(nospace3, trans[i]):
                                to_replace = re.search(nospace3, trans[i]).group(1)
                                trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", "_"))                                     
                            if re.search(interjections, trans[i]):
                                trans[i] = trans[i].replace(re.search(interjections, trans[i]).group(0), re.search(interjections, trans[i]).group(0).replace(" ", "_"))  
                            if re.search("(.* !)", trans[i]):
                                trans[i] = trans[i].replace(re.search("(.* !)", trans[i]).group(0), re.search("(.* !)", trans[i]).group(0).replace(" ", "") )                                
                                
                    elif 'colspan=' in li or '|||||||||||||||' in li:                  
                        # vu que les deux lignes translation et source remplissent cette condition, quand on arrive à source
                        # la string translation est remplie donc on n'écrase pas et on passe à source
                        if not translation:
                            translation = li.split('|')[-1]
                            for i in range(len(trans)): 
                                if any(ele in trans[i] for ele in nospace):
                                    trans[i] = trans[i].replace(" ", "_")     
                                if "jours de la semaine" or "jour de la semaine" in trans[i]:
                                    trans[i] = sansespace(trans[i]) 
                                
                        elif not source: # si déjà trans, alors on a la source
                            source = li.split('|')[-1].replace('[[','').replace(']]','')
                            if re.search("(\'\'.*\'\' ?,?).*", source):                               
                                other = re.search("(\'\'.*?\'\' ?,?).*", source).group(1)                                 
                                source = source.replace(other, '').replace("''","")                               
                            
                            if reLanguage.search(source):                              
                                grammatical = False     
                                break                                
                            if re.findall("''.*''", li):                 
                                if re.search(r"\)''", li):           
                                    location = re.findall(r"\(([^\)]+)\)", li)[0]
                                    if has_numbers(location):
                                        location = ''
                            if reDialect.search(li): 
                                dialect = reDialect.search(li)[0]                                
                            if ":" in source: 
                                l = re.search("(.*):.*", source).group(1) + ')'                                   
                            else: 
                                l = source
                            if re.search(r"(.*?)\[\[", li):                            
                                lang = re.search(r"(.*?)\[\[", li).group(0)
                                if reLanguage.search(lang): 
                                    grammatical = False                                                                                
                            if l in pages:                                     
                                if '#REDIRECTION' in pages[l]:                                 
                                    #on suit la redirection                                        
                                    newtitle = relinks.search(pages[l]).group(1)
                                    newtitle = newtitle[0].upper()+newtitle[1:]
                                    s = newtitle
                                    if newtitle in pages:
                                        bib_source = pages[newtitle]
                                        if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
                                            s += '(moyen breton)'                                           
                                else:
                                    s = l                                    
                                    bib_source = pages[l]
                                    if re.findall(r"\[\[Category:.*\|", bib_source):
                                        bibl = re.findall(r"\[\[Category:.*\|", bib_source)                   
                                        for ele in bibl:
                                            if reLanguage.search(ele):
                                                grammatical = False
                                                break
                                            tdialect = reDialect.search(ele)
                                            if tdialect:
                                                dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
                                            ttype = reType.search(ele)
                                            if ttype:
                                                texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "")                                                
                            else:                                    
                                s = l
                            if not dialect:                                           
                                if reDialect.search(li):
                                    dialect = reDialect.search(li).group(0)
                                else:                                                   
                                    for region in regions:                                                 
                                        if region in li:
                                            dialect = regions[region]                                             

            if not( tokens and trans and translation and source) and grammatical is not False: # if one is missing                
                if not tokens: 
                    errorout.write('\n'+currentPage+': Tokens line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True                    
                if not trans: 
                    errorout.write('\n'+currentPage+': Trans line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True
                if not translation:
                    errorout.write('\n'+currentPage+': Translation line could not be found:'+m.group(1)+'\n')
                    error_count += 1
                    problem = True
                if not source:
                    errorout.write('\n'+currentPage+': Source line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True
                continue
                
            if grammatical is False: 
                non_examples.write("__"+currentPage+"\nFound agrammatical sentence:\n"+ prettytable + "\n\n")
                other_table += 1 
                break  
                
            if comparison is True: 
                non_examples.write("__"+currentPage+"\nTableau de comparaison:\n"+ prettytable + "\n\n")
                other_table += 1 
                break                  
                

        # S'il y a plus de lignes que prévu : par ex une ligne de phonétique ou deux lignes source, ou les deux
        else:
            for li in m.group(1).split('\n'): # for every line of the table
                #si ce n'est pas une ligne de phonétique                

                if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==False:
                    if '[[*]]' in li.split('||')[0]:
                        grammatical = False
                        break
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    tokens = [cleanToken(t) for t in li.split('||')[1:]] 
                    if "vs." in tokens: 
                        comparison = True 
                    if tokens[0]=="[[*]]" :
                        grammatical = False
                        break      
                    if '' in tokens: 
                        while '' in tokens: 
                            tokens.remove('')  
                    if 'Equivalent [[KLT' in tokens[-1]: 
                        tokens[-1] = tokens[-1].replace('Equivalent [[KLT', '').strip()
                    if ']' in tokens[-1] and not '[…]' in tokens[-1] : 
                        tokens[-1] = tokens[-1].replace(']', '').strip()
                    if '[' in tokens[0] and not '[…]' in tokens[0]: 
                        tokens[0] = tokens[0].replace('[', '').strip()                        
                #si c'est une ligne de phonétique
                if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==True:
                    phonetic = [cleanToken(t) for t in li.split('||')[1:]]
                    clean_phonetic = ''
                    if '[' or '/' in phonetic[0]: 
                        phonetic[0] = phonetic[0].replace('[', '').replace('/', '').strip()         
                    if ']' or '/' in phonetic[-1]: 
                        phonetic[-1] = phonetic[-1].replace(']', '').replace('/', '').strip()
                    for ele in phonetic:        
                        clean_phonetic += ele
                        clean_phonetic += ' '

                if li[:3]=='|||':
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    # si on a rencontré une ligne phonétique et donc qu'on a pas encore de tokens, cette ligne (2e ligne) est la ligne de tokens                   
                    if not tokens :
                        tokens = [cleanToken(t) for t in li[3:].split('||')] 
                        if "vs." in tokens: 
                            comparison = True 
                        if '' in tokens: 
                            while '' in tokens: 
                                tokens.remove('') 
                        if 'Equivalent [[KLT' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace('Equivalent [[KLT', '').strip()
                        if 'Graphie peurunvan' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace('Graphie peurunvan', '').strip()
                        if ']' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace(']', '').strip()
                        if '[' in tokens[0]: 
                            tokens[0] = tokens[0].replace('[', '').strip() 
                        
                    elif not trans: #la première fois = la traduction  
                        if not "Équivalent standardisé" in li:                       
                            trans = [cleanToken(tr) for tr in li[3:].split('||')]                            
                            if '' in trans: 
                                while '' in trans: 
                                    trans.remove('')  
                            for i in range(len(trans)): 
                                if "aujourd'hui" in trans[i]:
                                    trans[i]  = trans[i].replace("aujourd'hui", "aujourdhui")      
                                if any(ele in trans[i] for ele in nospace):
                                    trans[i] = trans[i].replace(" ", "_")     
                                if "jours de la semaine" or "jour de la semaine" in trans[i]:
                                    trans[i] = sansespace(trans[i]) 
                                if re.search(interjections, trans[i]):
                                    trans[i] = trans[i].replace(re.search(interjections, trans[i]).group(0), re.search(interjections, trans[i]).group(0).replace(" ", "_"))
                                if re.search(r".*\|(.* !)\]\]", trans[i]): 
                                    to_replace = re.search(r".*\|(.* !)\]\]", trans[i]).group(1)
                                    trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", ""))    
                                if re.search(r".*\|(.* \?)\]\]", trans[i]): 
                                    to_replace2 = re.search(r".*\|(.* \?)\]\]", trans[i]).group(1)
                                    trans[i] = trans[i].replace(to_replace2, to_replace2.replace(" ", ""))  
                                    
                                    
                    elif 'colspan=' in li:
                        if li.startswith('|||colspan='):
                            translation = li.split('|')[-1]
                        elif not source:                        
                            source = li.split('|')[-1].replace('[[','').replace(']]','')   
                            if re.search("(\'\'.*\'\' ?,?).*", source):                              
                                other = re.search("(.*?\'\' ?,?).*", source).group(1)                               
                                source = source.replace(other, "").replace("''","")  
                            if reLanguage.search(li):
                                grammatical = False                             
                        elif not source2:                            
                            source2 = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'')    
                            if reLanguage.search(li):
                                grammatical = False 
                        if source:             
                            if source2 : 
                                #source = source2                                
                                source, source2 = source2, source
                            if re.findall("''.*''", li):                 
                                if re.search(r"\)''", li):           
                                    location = re.findall(r"\(([^\)]+)\)", li)[0]
                                    if has_numbers(location):
                                        location = ''
                            if reDialect.search(li): 
                                dialect = reDialect.search(li)[0]
                            if ":" in source: 
                                l = re.search("(.*):.*", source).group(1) + ')' 
                            else: 
                                l = source 
                            if re.search(r"(.*?)\[\[", li): 
                                lang = re.search(r"(.*?)\[\[", li).group(0)
                                if reLanguage.search(lang): 
                                    grammatical = False                                                          
                            if l in pages:                                 
                                if '#REDIRECTION' in pages[l]:                                       
                                    #on suit la redirection
                                    newtitle = relinks.search(pages[l]).group(1)                
                                    newtitle = newtitle[0].upper()+newtitle[1:]
                                    s = newtitle
                                    if newtitle in pages:
                                        bib_source = pages[newtitle]
                                        if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
                                            s += '(moyen breton)'                                           
                                else: 
                                    s = l                                              
                                    bib_source = pages[l]
                                if re.findall(r"\[\[Category:.*\|", bib_source):
                                    bibl = re.findall(r"\[\[Category:.*\|", bib_source)                 
                                    for ele in bibl:
                                        if reLanguage.search(ele):
                                            grammatical = False
                                            break
                                        tdialect = reDialect.search(ele)
                                        if tdialect:
                                            dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
                                        ttype = reType.search(ele)
                                        if ttype:
                                            texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "") 
                                if not dialect:
                                    if reDialect.search(li):
                                        dialect = reDialect.search(li).group(0)
                                    else:                                                
                                        for region in regions:
                                            if region in li:
                                                dialect = regions[region]
                                if reLanguage.search(li):
                                    grammatical = False                                                 
                            else: 
                                s = l
#à améliorer pour trouver le lieu                                
#                             if not location:
#                                 if re.search("(''.*'')", li):                                
#                                     test = re.search("''(.*)''", li).group(1) 
#                                     if not reDialect.search(test) and not test.endswith("ais"):                                     
#                                         location = test 
                           
            # print(tokens, trans, translation, source)
            if not( tokens and trans and translation and source) and grammatical is not False: # if one is missing                
                if not tokens: 
                    errorout.write('\n__'+currentPage+': Tokens line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not trans: 
                    errorout.write('\n__'+currentPage+': Trans line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not translation:
                    errorout.write('\n__'+currentPage+': Translation line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not source:
                    errorout.write('\n__'+currentPage+': Source line could not be found:'+m.group(1)+'\n')  
                    error_count += 1 
                continue 
            if grammatical is False: 
                non_examples.write("__"+currentPage+"\nFound agrammatical sentence:\n"+ prettytable + "\n\n")
                other_table += 1 
            if comparison is True: 
                non_examples.write("__"+currentPage+"\nTableau de comparaison:\n"+ prettytable + "\n\n")
                other_table += 1 

In [ ]:
#pour supprimer les mots agrammaticaux 
    #exemple: ( C'hwi / * Ac'hanoc'h ) --> C'hwi  
        cl_tokens = []   
       
        for i in range(len(tokens)):
            if tokens[i] == ']':
                tokens[i] = tokens[i].replace("]", "")     
            if tokens[i] == '[':
                tokens[i] = tokens[i].replace("[", "")  
            if '[…]' in tokens[i]:
                tokens[i] = tokens[i].replace('[…]', '')
            if '(…)' in tokens[i]:
                tokens[i] = tokens[i].replace('(…)', '')   
                if 'Equivalent [[KLT' in tokens[i]: 
                    tokens[i] = tokens[i].replace('Equivalent [[KLT', '').strip()
            if re.match(r".*\((.*\.)\)", title):            
                ele = re.match(r".*\((.*)\.\)", title).group(1)                
                tokens[i] = tokens[i].replace(ele, "")                
            if '/' in tokens[i] and '*' in tokens[i] and len(tokens)== len(trans):                
                elems = tokens[i].split('/')                
                for m in range(len(elems)):                    
                    idx = m                    
                    if '*' in elems[m]:                        
                        tokens[i] = tokens[i].replace(elems[m], '').replace('(', '').replace('{', '').replace('/', '')   
                        cl_tokens.append(tokens[i])                        
                        for i in range(len(trans)) :              
                            if '/' in trans[i]:                       
                                trans[i] = trans[i].split('/')[idx].replace('(', '').replace('{', '').replace('/', '')
            elif '/' in tokens[i] and not '*' in tokens[i] and len(tokens) == len(trans):           
                if reAlternatives.search(tokens[i]):  
                    try:                     
                        tokens[i] = tokens[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')
                        trans[i] = trans[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')   
                    except: 
                        errorout.write(f"__{currentPage}\nList index out of range, could not get first alternative:\ntoken:{tokens[i]}, trans: {trans[i]}\n\n")
                        error_count += 1
                        problem = True                                       
                        error_count += 1 
        if 'Équivalent standardisé' in tokens: 
            tokens.remove('Équivalent standardisé')        
        if re.search(r"\(.*/", translation): 
            options = re.search(r"\(.*\)", translation).group(0)
            option = re.search(r"\((.*?)/", translation).group(1)  
            translation = translation.replace(options, option)     
            
        if len(' '.join(tokens).split()) == len(' '.join(trans).split()): 
            tokens = ' '.join(tokens).split()               
            trans = ' '.join(trans).split()
            
            

        if grammatical is False: 
            break 
        text = ' '.join(tokens)
        text_ch = remettretoken(text) 


        if '' in tokens:          
            while '' in tokens: 
                tokens.remove('')            
        if '' in trans: 
            while '' in trans: 
                trans.remove('') 

In [ ]:
# construction of the conll:         
        text = ' '.join(tokens)
        text_ch = remettretoken(text)        
        references[source] = references.get(source,[])
        if text in references[source]:
            ind = references[source].index(text)+1
        else:
            references[source]+=[text]
            ind = 1
        sentences[text] = sentences.get(text,0)+1
        conllis = ['# sent_id = '+source.replace(' ', '') +'__'+str(ind)]
        conllis += ['# text = '+text_ch]        
        conllis += ['# text_fr = '+ remettretoken(cleanToken(translation)).replace('[…]', '').replace('(…)', '')]
        if phonetic:
            conllis += ['# text_phon = '+remettretoken(clean_phonetic.replace('[…]', '').replace('(…)', ''))]
        if dialect:
            conllis += ['# dialect = ' + dialect]
        if location:
            conllis += ['# location = ' + location]
        if s :
            conllis += ['# source = ' + s] 
        if source2:
            conllis += ['# source2 = ' + source2]
        if texttype :
            conllis += ['# texttype = ' + texttype]  

In [ ]:
##si on a trouvé du vert alors conllis += ['# phonetic = yes ']
        ##si un morceau en phonétique alors mettre en trait MISC

        extrai = 1
        
        if len(tokens) != len(trans):         
            errorout.write('\n___ '+ currentPage + "\n" + "len tokens:" 
            + str(len(tokens)) + "\nlen trans:" + str(len(trans)) + "\n" + "tokens:" + str(tokens)+ "\n"+
            "trans:" + str(trans) + "\n " +prettytable + "\n\n" )
            error_count += 1 
            continue    
        for i,t in enumerate(tokens):           
            if not t:
                errorout.write('\n___ '+currentPage+': pas de token '+str(i)+' in ' + prettytable + "\n")
                t='???'
                error_count += 1 
            if "[[R]]_[[4]] / [[+C]]_" in trans[i]: 
                trans[i] = trans[i].replace(" ", "").replace(" / ", ",")
                print(f"here: {trans[i]}")
            if len(trans)<=i:                
                errorout.write('\n___ '+currentPage+': la longueur de la glose est inférieure à la longueur des tokens '+str(i)+' in ' + prettytable + "\n")
                tr = '???'  
                error_count += 1 
            else:
                tr = trans[i]              
                
            # délimiteur : @ et espace         
            if t in one_token:             
                test1 = [t.replace(" ", "_").replace("-", "_")]  
                test2 = [tr.replace(" ", "_")]
            elif any(ele in t for ele in one_token):
                for i in range(len(one_token)):                
                    if re.search(one_token[i], t): 
                        if '-' in re.search(one_token[i], t).group(0):                        
                            test1 = re.split("@| |-", t)[:-2]                         
                            to_add1 = re.split("@| |-", t)[-2:]
                            test1.append('_'.join(to_add1))             
                            test2 = tr.split()[:-2]
                            to_add = tr.split()[-2:]
                            test2.append('_'.join(to_add))
                        elif '-' in re.search(one_token[i], t).group(0):                        
                            test1 = re.split("@| ", t)[:-2]                      
                            to_add1 = re.split("-| ", t)[-2:]
                            test1.append('_'.join(to_add1))              
                            test2 = tr.split()[:-2]
                            to_add = tr.split()[-2:]
                            test2.append('_'.join(to_add))                        
                       
            else: 
                test1 = re.split("@| ", t)
                if '' in test1:  
                    test1.remove("")           
                test2 = re.split(' |@', sansespace(tr))
                # si on a le même nombre d'éléments pour les tokens et la glose entre chaque || ||          
            if len(test1) == len(test2):                  
                for j,e in enumerate(test1):
                    t1 = test1[j]                  
                    tr1 = test2[j]                           
                    try:                    
                        if t1[-1] not in ".,;!?…" and t1 != "$$$":          
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)      
                            dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]# et morph ?
                            eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]             
                            if morph != '':
                                if 'R' in morph:                                
                                    eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]   
                                else: 
                                    eles[9]+='|'+morph                                
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)]                              
                            else: 
                                errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                    
                                error_count += 1
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1  
                            
                        elif t1[-1] in ".,;!?…" and not t1[-3:] == "..." :
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1[:-1], title, currentPage)
                            #si on ne trouve pas de pos, on cherche dans la page elle-même
                            if pos=='?' and t1[:-1].lower()==title.lower() and pagepos:
                                pos = pagepos
                            dic[t1[:-1]] = dic.get(t1[:-1], [])+[(lem,pos,gloss)]
                            eles = [str(extrai),remettretoken(t1[:-1]), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]
                            if morph != '':
                                eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]          
                            conllis += ['\t'.join(eles)]
                            extrai += 1
                            dic[t1[-1:]] = dic.get(t1[-1:], [(t1[-1:], 'PUNCT','punct')])
                            eles = [str(extrai),remettretoken(t1[-1:]), remettretoken(t1[-1:]), 'PUNCT']+5*['_']+['Gloss=punct']
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)]
                            else: 
                                print(f"error cas 2: ligne du conll invalide (longueur {len(eles)})")  
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1

                        else: 
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)
                            dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
                            eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]                       
                            if morph != '':
                                try:                             
                                    eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
                                except: 
                                    print("error") 
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)] 
                            else: 
                                errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                             
                                error_count += 1
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1
              
                    except:
                        continue                        
                        
            # si on a le nombre d'éléments de tokens est supérieur à celui de la glose                  
            elif len(test1)>len(test2):
                for j,e in enumerate(test1):
                    t1 = test1[j]                       
                    if t1 == "$$$":
                        errorout.write(f"__{currentPage}\nElément agrammatical dans la phrase:\n{prettytable}test1: {test1}\ntest2: {test2}\n\n")  
                        error_count += 1 
                    try:                    
                        tr1 = test2[j]
                    except:
                        tr1 = "?"                     
                        
                        
                    # si la ponctuation est tout simplement séparée du token                   
                    if t1 in "/-.,;!?…":
                        dic[t1] = dic.get(t1, [(t1, 'PUNCT','punct')])
                        eles = [str(extrai),t1, t1, 'PUNCT']+5*['_']+['Gloss=punct']
                        if len(eles) == 10:                              
                            conllis += ['\t'.join(eles)]
                        else: 
                            errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                                                    
                            error_count += 1
                        extrai += 1           
                    
                    # les autres tokens sont traités normalement
                    else:
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)
                        dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
                        eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]                       
                        if morph != '':
                            try:                             
                                eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
                            except: 
                                print("error") 
                        if len(eles) == 10:                              
                            conllis += ['\t'.join(eles)] 
                        else: 
                            errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                                                       
                            error_count += 1
                        if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                        extrai += 1

            # si on a le nombre d'éléments de la glose est supérieur à celui des tokens 
            elif len(test1)<len(test2):                   
                # print(f"PROBLEME ! éléments de la glose supérieur à celui des tokens: {test1, test2}") 
                    errorout.write('\n___ '+currentPage+': slash trouvé ' +str(i)+' in '+ 
prettytable + 'test1: ' + str(test1) + str(len(test1)) + "\n" + 'test2: ' + str(test2) + str(len(test2)) + "\n\n")
                    error_count += 1 
                    problem = True

In [ ]:
# pour vérifier qu'aucun token ne manque dans le conll
# problème : on splitte seulement sur les espaces mais il faudrait aussi splitter
# sur les apostrophes quand ils ne sont pas en début de mot 
        # test_conll = []
        # punct_list = [".", ",", ":", "!", ";", "…", ]
        # for i in range(len(conllis)):
        #     if re.search("^[0-9]", conllis[i]):
        #         test_conll.append(conllis[i].split('\t')[1])    
        # for ele in conllis[1].split()[3:]:
        #     if ele.replace(".", "").replace(":", "").replace(",", "").replace("!", "").replace("?", "").replace("…", "") not in test_conll and ele not in punct_list:
        #         print(f"error: le mot {ele} a disparu")
     
        if any("# dialect" in string for string in conllis):         
            for d in dialects:
                if f'# dialect = {d.capitalize()}' in conllis: 
                    dialects[d] += [conllis]  
        else: 
            dialects['inconnu'] +=  [conllis]             
    bilan.write(f"Nombre total: {total_example}, autres exemples: {other_table}, tableaux erreur {error_count}\n\n")
    return dialects

# trying to run through the whole site:

In [16]:
print(len(pages))
dic = {}
errorout = open('breton.errors.txt','w', encoding="utf-8")
nopos = open('no_pos.txt', 'w', encoding='utf-8')
pos_list = open('pos_list.txt','w', encoding='utf-8')
bilan = open('bilan.txt','w', encoding='utf-8')
doublons = open("doublon.txt", "w", encoding='utf-8')

for title in tqdm.tqdm(pages):
    analyzePage(title)  
for d in dialects:
    for conll in sorted(dialects[d], key = lambda x: x[1]):
        if conll[1].translate(str.maketrans('','',string.punctuation)) not in unique_sentences: 
            open('bretonconlls/'+re.sub(r'\W','_',d)+'.conllu','a', encoding="utf-8").write('\n'.join(conll) + '\n\n')
            unique_sentences.add(conll[1].translate(str.maketrans('','',string.punctuation)))
            converted_tables += 1  
        else:
            doublon += 1
            doublons.write("__\n" + str('\n'.join(conll)) + "\n\n")         
for key, value in lispos.items():
    pos_list.write(f"{key} \t {value}\n")  

9106


  4%|▍         | 380/9106 [00:00<00:12, 724.77it/s]

error


  5%|▍         | 455/9106 [00:00<00:19, 451.18it/s]

error


...

 91%|█████████ | 8257/9106 [00:14<00:01, 523.59it/s]

error
error
error


 99%|█████████▉| 9030/9106 [00:15<00:00, 681.10it/s]

error
error
error


100%|██████████| 9106/9106 [00:16<00:00, 564.91it/s]


In [15]:
"""
Exemple de tableau standard (4 lignes):

{| class="prettytable"
|(1)|| N'eo || ket || dleet || koduiñ || hag || evañ || er || memes || amzer.
|-
||| [[ne]] [[COP|est]] || [[ket|pas]] || [[dleout|dû]] || conduire || [[&|et]] || [[evañ|boire]] || [[P.e|en]].[[art|le]] || [[memes|même]] || temps
|-
|||colspan="10" | 'On ne doit pas boire et conduire en même temps /Il ne faut pas boire et conduire' 
|-
|||||||||colspan="10" | ''Lesneven/Kerlouan'', [[Y. M. (04/2016)]]
|}
"""
#dictionnaire des dialectes à récupérer
dialects = {"léonard" : [], "cornouaillais" : [], "trégorrois": [], "vannetais" : [], "standard" : [], "inconnu" : [],
            "breton central" : []}
references = {} # keeps track of the references of the sentences: ref:[sentence]
sentences = {} # keeps the actual texts
dic = {} # token to lemma, pos, gloss
lispos = {}
#set pour éviter les doublons
unique_sentences = set()
currentPage=''


## OUVERTURE DES FICHIERS
#fichier d'erreur
errorout = open('breton.errors.txt','w', encoding='utf-8')
bilan = open('bilan.txt','w', encoding='utf-8')
#fichier contenant les tableaux qu'on ne veut pas traiter
non_examples = open('non_examples.txt','w', encoding='utf-8')


### EXPRESSIONS REGULIERES
repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE)
rePOS = re.compile(r'verbes|auxiliaires|copules|adverbes|complémenteurs|conjonctions|prépositions|adjectifs|noms|particules verbales|interjections|postpositions|déterminants|quantifieurs|pronoms|noms propres|suffixe|interrogatifs|préfixes|modaux|pluriels|indéfinis|particules de discours|finales|exclamatifs', re.IGNORECASE)
reDialect = re.compile(r'léonard|cornouaillais|trégorrois|vannetais|breton central|standard', re.IGNORECASE)
reLanguage = re.compile(r"basque|français de Basse-Bretagne|franco-breton|gaulois|gallo|tohono o'odham|chalcatongo mixtec|tchèque|gallois|roumain|italien|espagnol|hébreu|arabe|français|anglais|allemand|moyen breton|vieux breton|breton pré-moderne", re.IGNORECASE)
reType = re.compile(r'ouvrages de recherche|références de corpus|élicitations|ouvrages pédagogiques|dictionnaires|grammaires', re.IGNORECASE)
reNoms = re.compile(r'Yann|Marijo|Anna|Angela|Alejandro|Odile|Anastazi|Nolwenn|Solange|Bastien|Soazig|Marie|Marsel|Lukaz|Lenaig|Kristof|Julie|Jenovefa|Simone|Mona|Mari', re.IGNORECASE)
relinks = re.compile(r"\[\[(.*?)\]\]")
relinks2 = re.compile(r"\[\[.*\]\]") 
reAlternatives = re.compile(r'\([^\W\d_].* */ *[^\W\d_].*\)', re.U)
reWord = re.compile(r'\[\[[a-z]+')
nospace2 = re.compile(r".*(-.*\(.*)")
nospace3 = re.compile(r".*(.*-,.*-).*")


#éléments de la glose dont on veut supprimer les espaces (pour éviter de splitter sur les espaces)
nospace = {"numéraux cardinaux": "numéraux_cardinaux","nom propre": "nom_propre" , "pronom incorporé": "pronom_incorporé" ,
"nom de titre": "nom_de_titre", 'pronom réfléchi': "pronom_réfléchi", 
"nom nu": "nom_nu", "pluriel interne": "pluriel_interne", "Les pronoms d'@incise contrastifs": "Les_pronoms_d'@incise_contrastifs", 
"particule o": "particule_o", "auxiliaire ober": "auxiliaire_ober" , "IMP on": "IMP_on",
"bezañ préverbal": "bezañ_préverbal", "Nep X":  "Nep_X", "Objet postverbal d'@un infinitif": "Objet_postverbal_d'un_infinitif", 
"Objet postverbal avec le verbe '@avoir": "Objet_postverbal_avec_le_verbe_'avoir", 
"Indéfini de choix libre par reduplication": "Indéfini_de_choix_libre_par_reduplication", 
"châtaigne électrique": "châtaigne_électrique", "numéral ordinal": "numéral_ordinal",
"avant de": "avant_de", "Pronom réfléchi": "Pronom_réfléchi", "Gwazh, gwashañ": "Gwazh,_gwashañ",
"jours de la semaine" : "jours_de_la_semaine", "jour de la semaine": "jour_de_la_semaine" } 

#catégories renseignées entre parenthèses dans la glose. Ex: -et(Adj.)
categories = {"V": "verbe", "Adj": "adjectif", "PL": "suffixe", "N": "nom", "Adv": "adverbe"}
# dictionnaire de correspondance pour trouver les dialectes dans les lignes source: Léon --> Léonard 
regions = {"Léon": "Léonard", "Cornouailles": "Cornouaillais", "Vannes": "Vannetais", "Trégor": "Trégorrois" }
# regex pour supprimer les espaces dans les interjections dans la glose
interjections = re.compile("Eh bien quoi|Ma parole|eh bien", re.IGNORECASE)
# Rannig + verbe avoir à ne pas séparer 
one_token = ["am-oa","he-devoa", "en em", "on-oa", "em-eus", "am-eus", "e-nije", "e-nevoa", "he-doa", 
             "en ur", "'m eus", "em-bije", "en-deus", "o-deus", "em-bije", "em-oa", "am-bez", "am-bo", "o-devoa", "o-devez", "o-deveze",
                        "o-doa"]

                        
#compteurs: nombre de tableaux trouvés, nombre de tableaux dans le fichier d'erreur, nombre de tableaux à ne pas traiter, 
#nombre de tableaux traités et nombre de doublons
total_example = 0
error_count = 0
other_table = 0
converted_tables = 0
doublon = 0



# getFirstPos: RECHERCHE DE POS DANS UNE PAGE TITRE 
# la fonction cherche le pos dans toutes les pages selon le lemme donné et retourne le pos associé
# si aucun pos n'est trouvé, on met un point d'interrogation
def getFirstPos(title, gloss, currentPage):
    mpos=''  
    if not title: 
        return '?' 
    elif title.isupper() and title != 'R':
        mpos = title  
    # si on cherche la Page "a", on aura une catégorie erronée (préposition au lieu de particule verbale) 
    elif title == 'a' or title == 'e' and 'R' in gloss :
        mpos='particule_verbale'
    elif title == 'P.e' or title == 'P/e':     
        mpos='préposition'
    # on vérifie si la catégorie est présente dans la glose
    elif re.match(r".*\((.*\.)\)", title) and re.match(r".*\((.*\.)\)", title).group(1) in categories:                    
        test_pos = re.match(r".*\((.*\.)\)", title).group(1)                   
        if test_pos.replace(".", "") in categories:                       
            mpos = categories[test_pos.replace(".", "")]                 
    else:        
    #si le titre contient un tiret bas, on le remplace par un espace pour le trouver dans les titres de pages
        if '_' in title: 
            title = title.replace("_", " ")
        if '#' in title: 
            title = title.replace("#", "-")            
#sinon on met une majuscule à la première lettre du titre
        else:             
            title = title[0].upper()+title[1:]
    #si le titre est contenu dans le dictionnaire pages
        if title in pages:           
            wikicode=''
            # on traite les redirections vers d'autres pages
            if '#REDIRECTION' in pages[title]:             
                #on suit la redirection
                newtitle = relinks.search(pages[title]).group(1)                
                newtitle = newtitle[0].upper()+newtitle[1:]
                if '_' in newtitle: 
                    newtitle = newtitle.replace("_", " ")            
                #et on regarde à nouveau si le titre est dans le dictionnaire pages
                if newtitle in pages:
                    wikicode = pages[newtitle]              
                else: 
                    newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
                    if newtitle in pages:
                        wikicode = pages[newtitle]
                    else:
                        newtitle = newtitle.split(',')[0]
                        newtitle = newtitle[0].upper()+newtitle[1:]
                        if newtitle in pages:
                            wikicode = pages[newtitle]
                        else:
                            errorout.write('__currentPage:\nstrange redirect to page that does not exist: '+newtitle+'\n')
                            error_count += 1                            
            else:
                wikicode = pages[title]   
            # on cherche les catégories renseignées sur la page
            if re.findall(r"\[\[Category:.*\|", wikicode):
                cats = re.findall(r"\[\[Category:.*\|", wikicode)               
                list_cats = set()
                for cat in cats:   
                    # on cherche les POS dans les catégories          
                    if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
                        list_cats.add(rePOS.search(cat).group(0).rstrip('s'))                    
                        if len(list_cats) == 1:                                          
                            mpos = list(list_cats)[0]                         
                        elif len(list_cats) >0: 
                            mpos = "/".join(list_cats)    
                    else: 
                        mpos="?"                              
            if mpos:              
                mpos = mpos
            else:
                mpos = "?"
        else:          
            if reNoms.search(title):             
                mpos = "nom_propre"
            # lorsqu'on se trouve sur la page du mot, le mot est entourés de trois guillemets (remplacés par %)
            # on cherche les POS sur la page où on se trouve
            elif "%" in title:             
                wikicode = pages[currentPage]                               
                if re.findall(r"\[\[Category:.*\|", wikicode):
                    cats = re.findall(r"\[\[Category:.*\|", wikicode)
                    list_cats = set()
                    for cat in cats:
                        if cat not in lispos: 
                            lispos[cat] = 0
                        else: 
                            lispos[cat] += 1                            
                        if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
                            list_cats.add(rePOS.search(cat).group(0).rstrip('s'))
                            if len(list_cats) == 1:                                          
                                mpos = list(list_cats)[0]
                            elif len(list_cats) >0: 
                                mpos = "/".join(list_cats)                               
            else:               
                mpos = "?"              
    return mpos


# Suppression des espaces dans les [[ ]] sinon, le split avec espace comme délimiteur espace va mal se passer...
# case : [[koll|perd]].[[-et (Adj.)|u]]
def sansespace(ch):
    for e in relinks.finditer(ch):
        if " " in e.group(0):
            start, end = e.start(), e.end()
            newch = re.sub(r"\[\[(.*?) (.*?)\]\]", "[[\\1_\\2]]", e.group(0))
            newch = newch.replace(' ', '_') 
            ch = ch[:start] + newch + ch[start+len(newch)+1:]
    return ch
#exemple :
#print(sansespace("[[art|le]] _[[1]]_[[hey j]]_[[maouez | femme]]"))
#resultat : "[[art|le]] _[[1]]_[[heyj]]_[[maouez| femme]]")
#print(sansespace("[[R]] [[+C]] [[ez eus|est]]"))


## REGEX POUR LES MUTATIONS ET CONSONNES EPENTHETIQUES
mutation1 = re.compile(r"_\[\[([12345])\]\]_\[\[(.*)\]\]")
mutation2 = re.compile(r"\[?\[?(.*)\]?\]?_\[\[([12345])\]\]_")

#ex: "[[ne]]_[[+C]]_"
epenthetique1 = re.compile(r"\[\[(.*)\]\]_,?/?(\[?\[?\+C\]?\]?)_")
#ex: [[ma|que]]_[[4]],+C_
epenthetique2 = re.compile(r"\[\[(.*)\]\]_\[\[([12345])\]\],?/?(\[?\[?\+C\]?\]?)_")
#ex: [[R]]_[[+C]],[[4]]_  
epenthetique3 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?),\[\[([12345])\]\]_")
#ex : [[R]]_[[+C]]_[[ez eus|est]]
#epenthetique4 = re.compile(r"\[\[(.*)\]\]_(\[?\[?\+C\]?\]?)_[\[(.*)\]\]") 
mutation = False
epenthetique = False

groupes_pipe = re.compile(r"(.*?)\|(.*)")



# tokentrans2lemposglossmorph: FONCTION QUI PREND UN TOKEN ET RETOURNE LEMME, POS, GLOSE
# NB : tr = traduction ; t = token

def tokentrans2lemposglossmorph(tr, t, title, currentPage):
    global error_count
    # print(f't:{t}, tr:{tr}')         
    lem, pos, gloss, morph = '', '', '', ''
    m = relinks.search(tr)     
    test_mutation1 = mutation1.search(tr)
    test_mutation2 = mutation2.search(tr)
    test_epenthetique1 = epenthetique1.search(tr)  
    test_epenthetique2 = epenthetique2.search(tr)  
    test_epenthetique3 = epenthetique3.search(tr)
    

    if test_mutation1 :     
        mutation = True        
        if ('.[[' or ']].' or '-[[' or ']]-' ) in tr:      
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')                 
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]]             
            if re.search(r"\[ ?- ?[A-Za-z]", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):         
                test = tr.split()[i]           
                if relinks2.search(test):                   
                    ele = relinks2.search(test).group(0)                                        
                    lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                    lems += [lem]
                    glosss += [gloss]
                    poss += [getFirstPos(lem, gloss, currentPage)]
                else:                                  
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                        error_count += 1
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."):
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    lemt.append(lems[i].replace('#', '-')) 
                    glosst.append(glosss[i])                    
                    post.append(poss[i]) 
        
        else:         
            morph = test_mutation1.group(1) + " " + "target"
            token_mute = test_mutation1.group(2)       
            token_mute_pipe = groupes_pipe.search(token_mute)
            if token_mute_pipe: 
                lem = token_mute_pipe.group(1)
                gloss = token_mute_pipe.group(2)
                if gloss == 'R':  
                    pos = "particule_verbale" 
                else:   
                    pos = getFirstPos(lem, gloss, currentPage)
            else :
                lem = t.lower()        
                gloss = token_mute   
                if gloss == 'R'  or '[[R]' in gloss:  
                    pos = "particule_verbale" 
                else:                          
                    pos = getFirstPos(t, gloss, currentPage)
                if pos == '?':
                    pos = getFirstPos(tr, gloss, currentPage)
                    
                    
    elif test_mutation2 :     
        mutation = True        
        if '.[[' or ']].' or '-[[' or ']]-' in tr:            
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')    
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]]             
            if re.search(r"\[ ?- ?[A-Za-z]", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):                
                test = tr.split()[i]  
                if relinks2.search(test):                 
                    ele = relinks2.search(test).group(0)                                        
                    try: 
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:                                   
                            errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                            error_count += 1
                else:                                  
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}") 
                        error_count += 1
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."): 
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    try:                     
                        lemt.append(lems[i].replace('#', '-')) 
                        glosst.append(glosss[i])                    
                        post.append(poss[i])
                    except:                        
                        errorout.write(f"__{currentPage}:\nErreur dans les morphèmes du mot suivant : {tr} et {t}")
                        error_count += 1
        
        else:              
            morph = test_mutation2.group(2) + " " + "trigger"
            token_mute = test_mutation2.group(1)       
            token_mute_pipe = groupes_pipe.search(token_mute)          
            if token_mute_pipe:      
                lem = token_mute_pipe.group(1)
                gloss = token_mute_pipe.group(2)
                if gloss == 'R'  or '[[R]' in gloss:  
                    pos = "particule_verbale" 
                else:                
                    pos = getFirstPos(lem, gloss, currentPage)
            else :
                lem = t.lower() 
                gloss = token_mute
                pos = getFirstPos(t, gloss, currentPage)
                if pos == '?':
                    pos = getFirstPos(tr, gloss,  currentPage)


#S'il y a une consonne épenthétique (+C)             
    elif test_epenthetique1 : 
        epenthetique = True    
        morph = test_epenthetique1.group(2) 
        token_mute = test_epenthetique1.group(1)       
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)

    elif test_epenthetique2 :
        epenthetique = True    
        morph = test_epenthetique2.group(2) + '|' +  test_epenthetique2.group(3) 
        token_mute = test_epenthetique2.group(1)    
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)
                             
    elif test_epenthetique3:
        epenthetique = True 
        morph = test_epenthetique3.group(2) + '|' +  test_epenthetique3.group(3)
        token_mute = test_epenthetique3.group(1)       
        token_mute_pipe = groupes_pipe.search(token_mute)
        if token_mute_pipe:      
            lem = token_mute_pipe.group(1)
            gloss = token_mute_pipe.group(2)
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale" 
            else:                
                pos = getFirstPos(lem, gloss, currentPage)
        else :
            lem = t.lower() 
            gloss = token_mute
            if gloss == 'R' or '[[R]' in gloss:  
                pos = "particule_verbale"                  
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)                    
                               
# si ce n'est pas une mutation et si on a des double crochets [[ ]]              
    elif m:     
        # cas "amalgames" et/ou mots contenant un tiret: 
        #exemple [[da|à]].[[pronom incorporé|vous]] ou [[den|parent]].[[pluriel interne|s]]-[[kozh|vieil]]  
        if '.[[' in tr or ']].' in tr or '-[['in tr or ']]-' in tr: 
            lems, glosss, poss, ponct = [], [], [], [] 
            lemt, glosst, post = [], [], []
            if '(Adj.)' or '(V.)' or '(PL.)' or 'P.e' or '(F.)' in tr:      
                tr = tr.replace('(Adj.)', '(Adj)').replace('(V.)', '(V)').replace('(PL.)', '(PL)').replace('P.e', 'P/e').replace('(F.)', '(F)').replace('(Adv.)', '(Adv)').replace('(M.)', '(M)').replace('(C.)', '(C)')      
            #s'il y a des tirets dans les groupes entre crochets, on les remplace par le signe #        
            #ex: [[genel|nat]].[[-idig|if]]  -->  [[genel|nat]].[[#idig|if]] 
            if re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr):               
                suffixe = re.search(r"\[ ?(-) ?[A-Za-z].*\|", tr).group(0)
                suffixe_transforme = suffixe.replace("-", "#")        
                tr = tr.replace(suffixe, suffixe_transforme)             
            tr = tr.replace(".", " .").replace("-", " -")                    
            for i in range(len(tr.split())):         
                test = tr.split()[i]                 
                if relinks2.search(test):                 
                    ele = relinks2.search(test).group(0)                                        
                    lem, pos, gloss, morph = tokentrans2lemposglossmorph(ele, t, title, currentPage)   
                    lems += [lem]
                    glosss += [gloss]
                    poss += [getFirstPos(lem, gloss, currentPage)]
                else:    
                    try:                    
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(test, t, title, currentPage)                 
                        lems += [lem]
                        glosss += [gloss]
                        poss += [getFirstPos(lem, gloss, currentPage)]
                    except:              
                        errorout.write(f"__{currentPage}:\nImpossible de trouver lem, pos, gloss, morph pour le mot suivant : {tr} et {t}")
                        error_count += 1 
                #on "recolle" les éléments avec le bon séparateur (tiret ou point)                                
                if tr.split()[i].startswith("."): 
                    lemt.append(f".{lems[i].replace('#', '-')}")
                    glosst.append(f".{glosss[i]}")  
                    post.append(f".{poss[i]}")  
                elif tr.split()[i].startswith("-"): 
                    lemt.append(f"-{lems[i].replace('#', '-')}")
                    glosst.append(f"-{glosss[i]}")  
                    post.append(f"-{poss[i]}")  
                else: 
                    lemt.append(lems[i].replace('#', '-')) 
                    glosst.append(glosss[i])                    
                    post.append(poss[i])                                        
            lem = ''.join(lemt)
            gloss = ''.join(glosst)
            pos = ''.join(post)             
            return lem, pos, gloss, morph        
        
        
        
        # cas "normal" : pas d'amalgame, un token et une gloss --> [[token|gloss]]
        if '|' in m.group(1):          
            if "numéraux_cardinaux" in m.group(1): 
                lem = t.lower() 
                pos = m.group(1).split('|')[0]
                gloss = m.group(1).split('|')[1] 
            else:                 
                lem = m.group(1).split('|')[0]
                gloss = m.group(1).split('|')[1]                  
                if gloss == 'R' or '[[R]' in gloss:  
                    pos = "particule_verbale"                      
                elif lem == 'P.e' or lem == 'P/e':  
                    lem = "e"
                    pos = "préposition"           
                elif lem == 'nom_propre':  
                    pos = lem
                    lem = gloss                 
                else:
                    pos = getFirstPos(lem, gloss, currentPage)                        
                if pos=='?':
                    pos = getFirstPos(t, gloss, currentPage)
                
        # cas sans amalgame mais sans gloss --> [[ne]]
        else:           
            lem = t
            gloss = relinks.search(tr).group(1)
            if gloss == 'R'  or '[[R]' in gloss:  
                pos = "particule_verbale"
            if 'DIM' in gloss and t.endswith('ig') or t.endswith('ig%'):
                lem = "-ig"                
                pos = "suffixe"                
            else:
                pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)      
    # cas d'un mot seul sans crochets --> foot dans la page "Abardaez,_enderv"      
    else :
        if "%" in t and t.replace("%", "").startswith(currentPage.lower()): 
            lem = currentPage.lower()        
        else: 
            lem = t
        gloss = tr
        if gloss == 'R'  or '[[R]' in gloss:  
            pos = "particule_verbale"            
        else:
            pos = getFirstPos(t, gloss, currentPage)
            if pos == '?':
                pos = getFirstPos(tr, gloss, currentPage)
    return lem, pos, gloss, morph


# Nettoyage des tokens + remplacement de certains caractères par d'autres
# pour éviter des erreurs quand on splitte
def cleanToken(t):
    t = t.replace("...", "…")   
    t = t.replace("tout.à.l'heure", "tout_à_l'heure")    
    t = t.replace("'''", "%")
    t = t.replace("''", "")
    t = re.sub(r'</?font.*?>','',t)
    t = re.sub(r'<sup>','_',t)
    t = re.sub(r'</sup>','_',t)
    t = re.sub(r'<u>','',t)
    t = re.sub(r'</u>','',t)
    t = re.sub(r'<sub>.*?</sub>','',t)
    t = re.sub(r'\(\[\[\*\]\].*?\)','$$$',t)
    t = re.sub(r"c'h",'cxxxh',t)
    t = re.sub(r"C'h",'Cxxxh',t)
    t = re.sub(r"`",'',t) 
    t = re.sub(r"#",'-',t)   
    # pour spliter sur les apostrophes sans les enlever : mettre @ pour spliter sur @ et donc conserver l'apostrophe
    if not t.strip().startswith("'"):   
        t = re.sub(r"'","'@",t)
    t = re.sub(r"bez'@","bez'",t) 
    t = re.sub(r"Bez'@","B ez'",t)     
    t = re.sub(r"d'@ici","d'ici",t)   
    t = re.sub("aujourd'@hui", "aujourd'hui", t)  
    t = re.sub("l'@heure", "l'heure", t)
    t = re.sub("si.ce.n'@est", "si.ce.n'est", t)   
    t = re.sub("aujourd'@hui", "aujourd'hui", t)      
    return t.strip()

# Remettre les tokens qui ont été remplacés
def remettretoken(t):
    t = re.sub(r"%","",t)
    t = re.sub(r'cxxxh',"c'h",t)
    t = re.sub(r'Cxxxh',"C'h",t)
    t = re.sub(r"'@", "'",t)
    t = re.sub(r"\$\$\$ ", "",t)
    t = re.sub(r"\[\[", "",t)   
    t = re.sub(r"\]\]", "",t) 
    t = t.replace("..", ".")     
    t = t.replace("e'@idin", "e'idin")          
    return t

# Savoir si une string contient un nombre
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

## analyzePage: FONCTION PRINCIPALE
def analyzePage(title): 
    global total_example, error_count, other_table, converted_tables, problem
    currentPage = title
    mpos = rePOS.search(pages[title])
    pagepos = None
    conlls=[]
    grammatical = True
    comparison = False  
    example = True
    s= ''
    if mpos:
        # met tout en minuscule
        pagepos = mpos.group(0).lower()
    #pour chaque tableau (et donc chaque exemple)
    nb_tables = 0    
    for m in repretty.finditer(pages[title]):
        total_example += 1        
        prettytable = m.group(1)        
        if "font color=white" in prettytable: 
            example = False
            non_examples.write("__"+currentPage+"\nAutre type de tableau:\n"+ prettytable + "\n\n")            
            break      
        nb_tables += 1        
        tokens, trans = [], []
        translation, source, source2 = '', '', ''
        location, dialect, phonetic, texttype, l = '','','', '', ''
        grammatical=True         

        # si le tableau est "standard" avec une ligne de token, une ligne de glose et une ligne source (9 lignes dans le texte wiki)
        if (len(m.group(1).split('\n'))) == 9 :
            for li in m.group(1).split('\n'): # for every line of the table              
                if li[:2]=='|(' :                   
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')                       
                    if '[[*]]' in li.split('||')[0]:
                        grammatical = False                     
                    tokens = [cleanToken(t) for t in li.split('||')[1:]]  
                    if "vs." in tokens: 
                        comparison = True                        
                    if tokens[0]=="[[*]]" :
                        grammatical = False   
                    elif '[' in tokens[0] and not "[…]" in tokens[0]:                     
                        tokens[0] = tokens[0].replace('[', '').strip()     
                    elif ']' in tokens[-1] and not '[…]' in tokens[-1]: 
                        tokens[-1] = tokens[-1].replace(']', '').strip()
                    if '' in tokens: 
                        while '' in tokens: 
                            tokens.remove('')         
                                                                           
                if li[:3]=='|||' and grammatical:
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    if not trans: #la première fois = la traduction                      
                        trans = [cleanToken(tr) for tr in li[3:].split('||')]                            
                        if '' in trans: 
                            while '' in trans: 
                                trans.remove('')                                
                        if "Glose en [[KLT']" in trans[-1]:
                            trans[-1] = trans[-1].replace("Glose en [[KLT']", '').strip()       
                        if 'Graphie peurunvan' in trans[-1]: 
                            trans[-1] = trans[-1].replace('Graphie peurunvan', '').strip()                           
                        for i in range(len(trans)):                         
                            for ele in nospace:                                
                                if ele in trans[i]: 
                                    trans[i]= trans[i].replace(ele, nospace[ele]) 
                            if re.search(nospace2, trans[i]):
                                to_replace = re.search(nospace2, trans[i]).group(1)
                                trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", "_")) 
                            if re.search(nospace3, trans[i]):
                                to_replace = re.search(nospace3, trans[i]).group(1)
                                trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", "_"))                                     
                            if re.search(interjections, trans[i]):
                                trans[i] = trans[i].replace(re.search(interjections, trans[i]).group(0), re.search(interjections, trans[i]).group(0).replace(" ", "_"))  
                            if re.search("(.* !)", trans[i]):
                                trans[i] = trans[i].replace(re.search("(.* !)", trans[i]).group(0), re.search("(.* !)", trans[i]).group(0).replace(" ", "") )                                
                                
                    elif 'colspan=' in li or '|||||||||||||||' in li:                  
                        # vu que les deux lignes translation et source remplissent cette condition, quand on arrive à source
                        # la string translation est remplie donc on n'écrase pas et on passe à source
                        if not translation:
                            translation = li.split('|')[-1]
                            for i in range(len(trans)): 
                                if any(ele in trans[i] for ele in nospace):
                                    trans[i] = trans[i].replace(" ", "_")     
                                if "jours de la semaine" or "jour de la semaine" in trans[i]:
                                    trans[i] = sansespace(trans[i]) 
                                
                        elif not source: # si déjà trans, alors on a la source
                            source = li.split('|')[-1].replace('[[','').replace(']]','')
                            if re.search("(\'\'.*\'\' ?,?).*", source):                               
                                other = re.search("(\'\'.*?\'\' ?,?).*", source).group(1)                                 
                                source = source.replace(other, '').replace("''","")                               
                            
                            if reLanguage.search(source):                              
                                grammatical = False     
                                break                                
                            if re.findall("''.*''", li):                 
                                if re.search(r"\)''", li):           
                                    location = re.findall(r"\(([^\)]+)\)", li)[0]
                                    if has_numbers(location):
                                        location = ''
                            if reDialect.search(li): 
                                dialect = reDialect.search(li)[0]                                
                            if ":" in source: 
                                l = re.search("(.*):.*", source).group(1) + ')'                                   
                            else: 
                                l = source
                            if re.search(r"(.*?)\[\[", li):                            
                                lang = re.search(r"(.*?)\[\[", li).group(0)
                                if reLanguage.search(lang): 
                                    grammatical = False                                                                                
                            if l in pages:                                     
                                if '#REDIRECTION' in pages[l]:                                 
                                    #on suit la redirection                                        
                                    newtitle = relinks.search(pages[l]).group(1)
                                    newtitle = newtitle[0].upper()+newtitle[1:]
                                    s = newtitle
                                    if newtitle in pages:
                                        bib_source = pages[newtitle]
                                        if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
                                            s += '(moyen breton)'                                           
                                else:
                                    s = l                                    
                                    bib_source = pages[l]
                                    if re.findall(r"\[\[Category:.*\|", bib_source):
                                        bibl = re.findall(r"\[\[Category:.*\|", bib_source)                   
                                        for ele in bibl:
                                            if reLanguage.search(ele):
                                                grammatical = False
                                                break
                                            tdialect = reDialect.search(ele)
                                            if tdialect:
                                                dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
                                            ttype = reType.search(ele)
                                            if ttype:
                                                texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "")                                                
                            else:                                    
                                s = l
                            if not dialect:                                           
                                if reDialect.search(li):
                                    dialect = reDialect.search(li).group(0)
                                else:                                                   
                                    for region in regions:                                                 
                                        if region in li:
                                            dialect = regions[region]                                             

            if not( tokens and trans and translation and source) and grammatical is not False: # if one is missing                
                if not tokens: 
                    errorout.write('\n'+currentPage+': Tokens line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True                    
                if not trans: 
                    errorout.write('\n'+currentPage+': Trans line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True
                if not translation:
                    errorout.write('\n'+currentPage+': Translation line could not be found:'+m.group(1)+'\n')
                    error_count += 1
                    problem = True
                if not source:
                    errorout.write('\n'+currentPage+': Source line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                    problem = True
                continue
                
            if grammatical is False: 
                non_examples.write("__"+currentPage+"\nFound agrammatical sentence:\n"+ prettytable + "\n\n")
                other_table += 1 
                break  
                
            if comparison is True: 
                non_examples.write("__"+currentPage+"\nTableau de comparaison:\n"+ prettytable + "\n\n")
                other_table += 1 
                break                  
                

        # S'il y a plus de lignes que prévu : par ex une ligne de phonétique ou deux lignes source, ou les deux
        else:
            for li in m.group(1).split('\n'): # for every line of the table
                #si ce n'est pas une ligne de phonétique                

                if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==False:
                    if '[[*]]' in li.split('||')[0]:
                        grammatical = False
                        break
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    tokens = [cleanToken(t) for t in li.split('||')[1:]] 
                    if "vs." in tokens: 
                        comparison = True 
                    if tokens[0]=="[[*]]" :
                        grammatical = False
                        break      
                    if '' in tokens: 
                        while '' in tokens: 
                            tokens.remove('')  
                    if 'Equivalent [[KLT' in tokens[-1]: 
                        tokens[-1] = tokens[-1].replace('Equivalent [[KLT', '').strip()
                    if ']' in tokens[-1] and not '[…]' in tokens[-1] : 
                        tokens[-1] = tokens[-1].replace(']', '').strip()
                    if '[' in tokens[0] and not '[…]' in tokens[0]: 
                        tokens[0] = tokens[0].replace('[', '').strip()                        
                #si c'est une ligne de phonétique
                if li[:2]=='|(' and bool(re.search(r'</?font.*?>',li))==True:
                    phonetic = [cleanToken(t) for t in li.split('||')[1:]]
                    clean_phonetic = ''
                    if '[' or '/' in phonetic[0]: 
                        phonetic[0] = phonetic[0].replace('[', '').replace('/', '').strip()         
                    if ']' or '/' in phonetic[-1]: 
                        phonetic[-1] = phonetic[-1].replace(']', '').replace('/', '').strip()
                    for ele in phonetic:        
                        clean_phonetic += ele
                        clean_phonetic += ' '

                if li[:3]=='|||':
                    if '|| ||' in li:
                        li = li.replace('|| ||', '||')
                    # si on a rencontré une ligne phonétique et donc qu'on a pas encore de tokens, cette ligne (2e ligne) est la ligne de tokens                   
                    if not tokens :
                        tokens = [cleanToken(t) for t in li[3:].split('||')] 
                        if "vs." in tokens: 
                            comparison = True 
                        if '' in tokens: 
                            while '' in tokens: 
                                tokens.remove('') 
                        if 'Equivalent [[KLT' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace('Equivalent [[KLT', '').strip()
                        if 'Graphie peurunvan' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace('Graphie peurunvan', '').strip()
                        if ']' in tokens[-1]: 
                            tokens[-1] = tokens[-1].replace(']', '').strip()
                        if '[' in tokens[0]: 
                            tokens[0] = tokens[0].replace('[', '').strip() 
                        
                    elif not trans: #la première fois = la traduction  
                        if not "Équivalent standardisé" in li:                       
                            trans = [cleanToken(tr) for tr in li[3:].split('||')]                            
                            if '' in trans: 
                                while '' in trans: 
                                    trans.remove('')  
                            for i in range(len(trans)): 
                                if "aujourd'hui" in trans[i]:
                                    trans[i]  = trans[i].replace("aujourd'hui", "aujourdhui")      
                                if any(ele in trans[i] for ele in nospace):
                                    trans[i] = trans[i].replace(" ", "_")     
                                if "jours de la semaine" or "jour de la semaine" in trans[i]:
                                    trans[i] = sansespace(trans[i]) 
                                if re.search(interjections, trans[i]):
                                    trans[i] = trans[i].replace(re.search(interjections, trans[i]).group(0), re.search(interjections, trans[i]).group(0).replace(" ", "_"))
                                if re.search(r".*\|(.* !)\]\]", trans[i]): 
                                    to_replace = re.search(r".*\|(.* !)\]\]", trans[i]).group(1)
                                    trans[i] = trans[i].replace(to_replace, to_replace.replace(" ", ""))    
                                if re.search(r".*\|(.* \?)\]\]", trans[i]): 
                                    to_replace2 = re.search(r".*\|(.* \?)\]\]", trans[i]).group(1)
                                    trans[i] = trans[i].replace(to_replace2, to_replace2.replace(" ", ""))  
                                    
                                    
                    elif 'colspan=' in li:
                        if li.startswith('|||colspan='):
                            translation = li.split('|')[-1]
                        elif not source:                        
                            source = li.split('|')[-1].replace('[[','').replace(']]','')   
                            if re.search("(\'\'.*\'\' ?,?).*", source):                              
                                other = re.search("(.*?\'\' ?,?).*", source).group(1)                               
                                source = source.replace(other, "").replace("''","")  
                            if reLanguage.search(li):
                                grammatical = False                             
                        elif not source2:                            
                            source2 = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'')    
                            if reLanguage.search(li):
                                grammatical = False 
                        if source:             
                            if source2 : 
                                #source = source2                                
                                source, source2 = source2, source
                            if re.findall("''.*''", li):                 
                                if re.search(r"\)''", li):           
                                    location = re.findall(r"\(([^\)]+)\)", li)[0]
                                    if has_numbers(location):
                                        location = ''
                            if reDialect.search(li): 
                                dialect = reDialect.search(li)[0]
                            if ":" in source: 
                                l = re.search("(.*):.*", source).group(1) + ')' 
                            else: 
                                l = source 
                            if re.search(r"(.*?)\[\[", li): 
                                lang = re.search(r"(.*?)\[\[", li).group(0)
                                if reLanguage.search(lang): 
                                    grammatical = False                                                          
                            if l in pages:                                 
                                if '#REDIRECTION' in pages[l]:                                       
                                    #on suit la redirection
                                    newtitle = relinks.search(pages[l]).group(1)                
                                    newtitle = newtitle[0].upper()+newtitle[1:]
                                    s = newtitle
                                    if newtitle in pages:
                                        bib_source = pages[newtitle]
                                        if "moyen breton" in str(re.findall(r"\[\[Category:.*\|", bib_source)):
                                            s += '(moyen breton)'                                           
                                else: 
                                    s = l                                              
                                    bib_source = pages[l]
                                if re.findall(r"\[\[Category:.*\|", bib_source):
                                    bibl = re.findall(r"\[\[Category:.*\|", bib_source)                 
                                    for ele in bibl:
                                        if reLanguage.search(ele):
                                            grammatical = False
                                            break
                                        tdialect = reDialect.search(ele)
                                        if tdialect:
                                            dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
                                        ttype = reType.search(ele)
                                        if ttype:
                                            texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "") 
                                if not dialect:
                                    if reDialect.search(li):
                                        dialect = reDialect.search(li).group(0)
                                    else:                                                
                                        for region in regions:
                                            if region in li:
                                                dialect = regions[region]
                                if reLanguage.search(li):
                                    grammatical = False                                                 
                            else: 
                                s = l
#à améliorer pour trouver le lieu                                
#                             if not location:
#                                 if re.search("(''.*'')", li):                                
#                                     test = re.search("''(.*)''", li).group(1) 
#                                     if not reDialect.search(test) and not test.endswith("ais"):                                     
#                                         location = test 
                           
            # print(tokens, trans, translation, source)
            if not( tokens and trans and translation and source) and grammatical is not False: # if one is missing                
                if not tokens: 
                    errorout.write('\n__'+currentPage+': Tokens line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not trans: 
                    errorout.write('\n__'+currentPage+': Trans line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not translation:
                    errorout.write('\n__'+currentPage+': Translation line could not be found:'+m.group(1)+'\n')
                    error_count += 1 
                if not source:
                    errorout.write('\n__'+currentPage+': Source line could not be found:'+m.group(1)+'\n')  
                    error_count += 1 
                continue 
            if grammatical is False: 
                non_examples.write("__"+currentPage+"\nFound agrammatical sentence:\n"+ prettytable + "\n\n")
                other_table += 1 
            if comparison is True: 
                non_examples.write("__"+currentPage+"\nTableau de comparaison:\n"+ prettytable + "\n\n")
                other_table += 1 


    #pour supprimer les mots agrammaticaux 
    #exemple: ( C'hwi / * Ac'hanoc'h ) --> C'hwi  
        cl_tokens = []   
       
        for i in range(len(tokens)):
            if tokens[i] == ']':
                tokens[i] = tokens[i].replace("]", "")     
            if tokens[i] == '[':
                tokens[i] = tokens[i].replace("[", "")  
            if '[…]' in tokens[i]:
                tokens[i] = tokens[i].replace('[…]', '')
            if '(…)' in tokens[i]:
                tokens[i] = tokens[i].replace('(…)', '')   
                if 'Equivalent [[KLT' in tokens[i]: 
                    tokens[i] = tokens[i].replace('Equivalent [[KLT', '').strip()
            if re.match(r".*\((.*\.)\)", title):            
                ele = re.match(r".*\((.*)\.\)", title).group(1)                
                tokens[i] = tokens[i].replace(ele, "")                
            if '/' in tokens[i] and '*' in tokens[i] and len(tokens)== len(trans):                
                elems = tokens[i].split('/')                
                for m in range(len(elems)):                    
                    idx = m                    
                    if '*' in elems[m]:                        
                        tokens[i] = tokens[i].replace(elems[m], '').replace('(', '').replace('{', '').replace('/', '')   
                        cl_tokens.append(tokens[i])                        
                        for i in range(len(trans)) :              
                            if '/' in trans[i]:                       
                                trans[i] = trans[i].split('/')[idx].replace('(', '').replace('{', '').replace('/', '')
            elif '/' in tokens[i] and not '*' in tokens[i] and len(tokens) == len(trans):           
                if reAlternatives.search(tokens[i]):  
                    try:                     
                        tokens[i] = tokens[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')
                        trans[i] = trans[i].split('/')[0].replace('(', '').replace('{', '').replace('/', '')   
                    except: 
                        errorout.write(f"__{currentPage}\nList index out of range, could not get first alternative:\ntoken:{tokens[i]}, trans: {trans[i]}\n\n")
                        error_count += 1
                        problem = True                                       
                        error_count += 1 
        if 'Équivalent standardisé' in tokens: 
            tokens.remove('Équivalent standardisé')        
        if re.search(r"\(.*/", translation): 
            options = re.search(r"\(.*\)", translation).group(0)
            option = re.search(r"\((.*?)/", translation).group(1)  
            translation = translation.replace(options, option)     
            
        if len(' '.join(tokens).split()) == len(' '.join(trans).split()): 
            tokens = ' '.join(tokens).split()               
            trans = ' '.join(trans).split()
            
            

        if grammatical is False: 
            break 
        text = ' '.join(tokens)
        text_ch = remettretoken(text) 


        if '' in tokens:          
            while '' in tokens: 
                tokens.remove('')            
        if '' in trans: 
            while '' in trans: 
                trans.remove('') 
        # construction of the conll:         
        text = ' '.join(tokens)
        text_ch = remettretoken(text)        
        references[source] = references.get(source,[])
        if text in references[source]:
            ind = references[source].index(text)+1
        else:
            references[source]+=[text]
            ind = 1
        sentences[text] = sentences.get(text,0)+1
        conllis = ['# sent_id = '+source.replace(' ', '') +'__'+str(ind)]
        conllis += ['# text = '+text_ch]        
        conllis += ['# text_fr = '+ remettretoken(cleanToken(translation)).replace('[…]', '').replace('(…)', '')]
        if phonetic:
            conllis += ['# text_phon = '+remettretoken(clean_phonetic.replace('[…]', '').replace('(…)', ''))]
        if dialect:
            conllis += ['# dialect = ' + dialect]
        if location:
            conllis += ['# location = ' + location]
        if s :
            conllis += ['# source = ' + s] 
        if source2:
            conllis += ['# source2 = ' + source2]
        if texttype :
            conllis += ['# texttype = ' + texttype]   
        ##si on a trouvé du vert alors conllis += ['# phonetic = yes ']
        ##si un morceau en phonétique alors mettre en trait MISC

        extrai = 1
        
        if len(tokens) != len(trans):         
            errorout.write('\n___ '+ currentPage + "\n" + "len tokens:" 
            + str(len(tokens)) + "\nlen trans:" + str(len(trans)) + "\n" + "tokens:" + str(tokens)+ "\n"+
            "trans:" + str(trans) + "\n " +prettytable + "\n\n" )
            error_count += 1 
            continue    
        for i,t in enumerate(tokens):           
            if not t:
                errorout.write('\n___ '+currentPage+': pas de token '+str(i)+' in ' + prettytable + "\n")
                t='???'
                error_count += 1 
            if "[[R]]_[[4]] / [[+C]]_" in trans[i]: 
                trans[i] = trans[i].replace(" ", "").replace(" / ", ",")
                print(f"here: {trans[i]}")
            if len(trans)<=i:                
                errorout.write('\n___ '+currentPage+': la longueur de la glose est inférieure à la longueur des tokens '+str(i)+' in ' + prettytable + "\n")
                tr = '???'  
                error_count += 1 
            else:
                tr = trans[i]              
                
            # délimiteur : @ et espace         
            if t in one_token:             
                test1 = [t.replace(" ", "_").replace("-", "_")]  
                test2 = [tr.replace(" ", "_")]
            elif any(ele in t for ele in one_token):
                for i in range(len(one_token)):                
                    if re.search(one_token[i], t): 
                        if '-' in re.search(one_token[i], t).group(0):                        
                            test1 = re.split("@| |-", t)[:-2]                         
                            to_add1 = re.split("@| |-", t)[-2:]
                            test1.append('_'.join(to_add1))             
                            test2 = tr.split()[:-2]
                            to_add = tr.split()[-2:]
                            test2.append('_'.join(to_add))
                        elif '-' in re.search(one_token[i], t).group(0):                        
                            test1 = re.split("@| ", t)[:-2]                      
                            to_add1 = re.split("-| ", t)[-2:]
                            test1.append('_'.join(to_add1))              
                            test2 = tr.split()[:-2]
                            to_add = tr.split()[-2:]
                            test2.append('_'.join(to_add))                        
                       
            else: 
                test1 = re.split("@| ", t)
                if '' in test1:  
                    test1.remove("")           
                test2 = re.split(' |@', sansespace(tr))
            # si on a le même nombre d'éléments pour les tokens et la glose entre chaque || ||          
            if len(test1) == len(test2):                  
                for j,e in enumerate(test1):
                    t1 = test1[j]                  
                    tr1 = test2[j]                           
                    try:                    
                        if t1[-1] not in ".,;!?…" and t1 != "$$$":          
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)      
                            dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]# et morph ?
                            eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]             
                            if morph != '':
                                if 'R' in morph:                                
                                    eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]   
                                else: 
                                    eles[9]+='|'+morph                                
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)]                              
                            else: 
                                errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                    
                                error_count += 1
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1  
                            
                        elif t1[-1] in ".,;!?…" and not t1[-3:] == "..." :
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1[:-1], title, currentPage)
                            #si on ne trouve pas de pos, on cherche dans la page elle-même
                            if pos=='?' and t1[:-1].lower()==title.lower() and pagepos:
                                pos = pagepos
                            dic[t1[:-1]] = dic.get(t1[:-1], [])+[(lem,pos,gloss)]
                            eles = [str(extrai),remettretoken(t1[:-1]), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]
                            if morph != '':
                                eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]          
                            conllis += ['\t'.join(eles)]
                            extrai += 1
                            dic[t1[-1:]] = dic.get(t1[-1:], [(t1[-1:], 'PUNCT','punct')])
                            eles = [str(extrai),remettretoken(t1[-1:]), remettretoken(t1[-1:]), 'PUNCT']+5*['_']+['Gloss=punct']
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)]
                            else: 
                                print(f"error cas 2: ligne du conll invalide (longueur {len(eles)})")  
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1

                        else: 
                            lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)
                            dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
                            eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]                       
                            if morph != '':
                                try:                             
                                    eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
                                except: 
                                    print("error") 
                            if len(eles) == 10:                              
                                conllis += ['\t'.join(eles)] 
                            else: 
                                errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                             
                                error_count += 1
                            if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                            extrai += 1
              
                    except:
                        continue                        
                        
            # si on a le nombre d'éléments de tokens est supérieur à celui de la glose                  
            elif len(test1)>len(test2):
                for j,e in enumerate(test1):
                    t1 = test1[j]                       
                    if t1 == "$$$":
                        errorout.write(f"__{currentPage}\nElément agrammatical dans la phrase:\n{prettytable}test1: {test1}\ntest2: {test2}\n\n")  
                        error_count += 1 
                    try:                    
                        tr1 = test2[j]
                    except:
                        tr1 = "?"                     
                        
                        
                    # si la ponctuation est tout simplement séparée du token                   
                    if t1 in "/-.,;!?…":
                        dic[t1] = dic.get(t1, [(t1, 'PUNCT','punct')])
                        eles = [str(extrai),t1, t1, 'PUNCT']+5*['_']+['Gloss=punct']
                        if len(eles) == 10:                              
                            conllis += ['\t'.join(eles)]
                        else: 
                            errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                                                    
                            error_count += 1
                        extrai += 1           
                    
                    # les autres tokens sont traités normalement
                    else:
                        lem, pos, gloss, morph = tokentrans2lemposglossmorph(tr1, t1, title, currentPage)
                        dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
                        eles = [str(extrai),remettretoken(t1), remettretoken(lem), pos]+5*['_']+['Gloss='+ remettretoken(gloss)]                       
                        if morph != '':
                            try:                             
                                eles[9]+='|Mutation='+morph.split(" ")[1]+'|MutationNUM='+morph.split(" ")[0]
                            except: 
                                print("error") 
                        if len(eles) == 10:                              
                            conllis += ['\t'.join(eles)] 
                        else: 
                            errorout.write(f"\n___{currentPage}\n Erreur dans la : \n {prettytable}\n.Ligne du conll invalide (longueur {len(eles)})\n\n")                                                                       
                            error_count += 1
                        if not (lem or pos or gloss or morph) :
                                errorout.write(f"\n__{currentPage}\nImpossible de trouver lem, pos, gloss, morph\n{prettytable}\n\n")
                                error_count += 1
                        extrai += 1

            # si on a le nombre d'éléments de la glose est supérieur à celui des tokens 
            elif len(test1)<len(test2):                   
                # print(f"PROBLEME ! éléments de la glose supérieur à celui des tokens: {test1, test2}") 
                    errorout.write('\n___ '+currentPage+': slash trouvé ' +str(i)+' in '+ 
prettytable + 'test1: ' + str(test1) + str(len(test1)) + "\n" + 'test2: ' + str(test2) + str(len(test2)) + "\n\n")
                    error_count += 1 
                    problem = True

# pour vérifier qu'aucun token ne manque dans le conll
# problème : on splitte seulement sur les espaces mais il faudrait aussi splitter
# sur les apostrophes quand ils ne sont pas en début de mot 
        # test_conll = []
        # punct_list = [".", ",", ":", "!", ";", "…", ]
        # for i in range(len(conllis)):
        #     if re.search("^[0-9]", conllis[i]):
        #         test_conll.append(conllis[i].split('\t')[1])    
        # for ele in conllis[1].split()[3:]:
        #     if ele.replace(".", "").replace(":", "").replace(",", "").replace("!", "").replace("?", "").replace("…", "") not in test_conll and ele not in punct_list:
        #         print(f"error: le mot {ele} a disparu")
     
        if any("# dialect" in string for string in conllis):         
            for d in dialects:
                if f'# dialect = {d.capitalize()}' in conllis: 
                    dialects[d] += [conllis]  
        else: 
            dialects['inconnu'] +=  [conllis]             
    bilan.write(f"Nombre total: {total_example}, autres exemples: {other_table}, tableaux erreur {error_count}\n\n")
    return dialects